In [52]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.nn import functional as F

In [53]:
columns = ['Class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']

In [54]:
wine_data = pd.read_csv("data/Wine.csv", names=columns)
wine_data.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [55]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

wine_data["Class"] = le.fit_transform(wine_data["Class"])

In [56]:
wine_features = wine_data.drop("Class", axis=1)

In [57]:
wine_target = wine_data[["Class"]]
wine_target.sample(5)

,Class
49,0
166,2
78,1
89,1
2,0


In [58]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(wine_features, wine_target, test_size=0.2, random_state=42)

In [59]:
X_train.shape, y_train.shape

((142, 13), (142, 1))

In [60]:
Xtrain_ = torch.from_numpy(X_train.values).float()
Xtest_ = torch.from_numpy(X_test.values).float()

In [74]:
ytrain_ = torch.from_numpy(y_train.values).view(1, -1)[0]
ytest_ = torch.from_numpy(y_test.values).view(1, -1)[0]

In [68]:
input_size = 13
output_size = 3
hidden_size = 100

In [69]:
class StupidNet(nn.Module):
    def __init__(self):
        super(StupidNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, X):
        X = torch.sigmoid(self.fc1(X))
        X = torch.sigmoid(self.fc2(X))
        X = self.fc3(X)
        return F.log_softmax(X, dim=-1)

In [70]:
model = StupidNet()

import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.01)

loss_fn = nn.NLLLoss()

In [71]:
epochs = 1000

In [76]:
for epoch in range(epochs):
    
    optimizer.zero_grad()
    ypred_ = model(Xtrain_)
    
    loss = loss_fn(ypred_, ytrain_)
    
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        print("Epoch", epoch, "loss", loss.item())

Epoch 0 loss 1.1033815145492554
Epoch 100 loss 0.15045908093452454
Epoch 200 loss 0.521255612373352
Epoch 300 loss 0.14882370829582214
Epoch 400 loss 0.0654686838388443
Epoch 500 loss 0.05225425213575363
Epoch 600 loss 0.2822467088699341
Epoch 700 loss 0.2057887613773346
Epoch 800 loss 0.19417548179626465
Epoch 900 loss 0.06462456285953522


In [79]:
torch.save(model, "models/wine_classifier.pt")

/home/daniele/anaconda3/envs/ml/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type StupidNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [81]:
new_model = torch.load("models/wine_classifier.pt")

In [92]:
predict_out = new_model(Xtest_)
predict_y = torch.argmax(predict_out, 1)

In [93]:
from sklearn.metrics import accuracy_score

accuracy_score(ytest_.data, predict_y.data)

0.9444444444444444

In [100]:
torch.save(new_model.state_dict(), "models/wine_state_dict")

In [101]:
new_new_model = StupidNet()
new_new_model.load_state_dict(torch.load("models/wine_state_dict"))

In [105]:
new_new_model.eval()
predict_out = new_new_model(Xtest_)
predict_y = torch.argmax(predict_out, 1)

In [106]:
accuracy_score(ytest_.data, predict_y.data)

0.9444444444444444